In [1]:
%config Completer.use_jedi = False
import numpy as np
import xarray as xr
import datashader as ds
import pandas as pd
from datashader.transfer_functions import shade, Images
from datashader.colors import Elevation

#conda install may not work; used pip
from satsearch import Search

import xrspatial.multispectral as ms

In [32]:

items = Search(url="https://earth-search.aws.element84.com/v0",
    bbox=[-93.112301, 29.649001, -92.075965, 30.719868],
    collections=["sentinel-s2-l2a-cogs"],
    datetime="2020-01-01/2020-01-31").items()
# items.download_assets()

#print first item
print(items[0])
print(items[0].assets.keys())
print(items[0].assets['B02']['href'])

S2B_15RVN_20200131_0_L2A
dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/15/R/VN/2020/1/S2B_15RVN_20200131_0_L2A/B02.tif


In [4]:
tiles = ['15RWQ']

dates =[
  '20200119',
  '20200228',
  '20200418',
  '20200518',
  '20200617',
  '20200717',
  '20200915',
  '20201015',
  '20201104',
  '20201224',
]

dates_mapping = {v: i for i, v in enumerate(dates)}

bands = ['B02', 'B03', 'B04']
bands_mapping = {v: i for i, v in enumerate(bands)}

In [ ]:
path_dict = {}

for date in dates:
    path_dict[date] = {}
    for tile in tiles:
        path_dict[date][tile] = {}
        for band in bands:
            path_dict[date][tile][band] = f'sentinel-s2-l2a-cogs/2020-01-31/S2A_{tile}_{date}_0_L2A_{band}.tif'


def read_raw_data(path_dict=path_dict, dates=dates, tiles=tiles, bands=bands):   
    data_dict = {}
    
    for date in dates:
        data_dict[date] = {}
        
        for tile_id in tiles:            
            data_dict[date][tile_id] = {}
            for band in bands:
                # get the data path                        
                file_path = path_dict[date][tile_id][band]
                agg = xr.open_rasterio(file_path).astype(np.float)
                # correct value of band dim
                band_id = bands_mapping[band]
                agg['band'] = [band_id]
                data_dict[date][tile_id][band] = agg
    
    return data_dict


In [7]:
data_dict = read_raw_data()
data_dict

RasterioIOError: E:\S2A_15RWQ_20200119_0_L2A\B02.tif: No such file or directory

In [47]:
s2a_image_paths = ['S2A_15RWQ_20200129_0_L2A_B02.tif', 'S2A_15RWQ_20200129_0_L2A_B03.tif', 
             'S2A_15RWQ_20200129_0_L2A_B04.tif']

In [50]:
band_aggs = []
i = 1
for path in s2a_image_paths:
    full_path = f"sentinel-s2-l2a-cogs/2020-01-29/{path}"
    agg = xr.open_rasterio(full_path).astype(np.float)
    agg['band'] = [i]
    band_aggs.append(agg)
    i += 1
band_aggs

<ipython-input-50-7563be2b5e2b>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  agg = xr.open_rasterio(full_path).astype(np.float)


[<xarray.DataArray (band: 1, y: 10980, x: 10980)>
 array([[[7736., 7612., 7480., ..., 7052., 7140., 7260.],
         [7616., 7524., 7412., ..., 7280., 7352., 7432.],
         [7672., 7616., 7496., ..., 7516., 7432., 7536.],
         ...,
         [5504., 5444., 5340., ...,    0.,    0.,    0.],
         [5748., 5668., 5568., ...,    0.,    0.,    0.],
         [5928., 5824., 5724., ...,    0.,    0.,    0.]]])
 Coordinates:
   * band     (band) int64 1
   * y        (y) float64 3.5e+06 3.5e+06 3.5e+06 ... 3.39e+06 3.39e+06 3.39e+06
   * x        (x) float64 5e+05 5e+05 5e+05 ... 6.098e+05 6.098e+05 6.098e+05
 Attributes:
     transform:           (10.0, 0.0, 499980.0, 0.0, -10.0, 3500040.0)
     crs:                 +init=epsg:32615
     res:                 (10.0, 10.0)
     is_tiled:            1
     nodatavals:          (0.0,)
     scales:              (1.0,)
     offsets:             (0.0,)
     AREA_OR_POINT:       Area
     OVR_RESAMPLING_ALG:  AVERAGE,
 <xarray.DataArray (band: